<a href="https://colab.research.google.com/github/Ian3000Dias/AudioBook-Platfrom-Analysis/blob/main/AudioBook_DataScience_BusinessCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>